In [1]:
import sys
print(sys.executable)

c:\Users\It_nova\OneDrive - Conconcreto\Escritorio\proyectos_personales\celerix\credito-digital-analytics\venv\Scripts\python.exe


In [12]:
import pandas as pd, numpy as np, pathlib as pl 
raw_path = pl.Path('../data/raw/Base PRUEBA - ANALITICA.xlsx')
df = pd.read_excel(raw_path, sheet_name='BD')
df.head()

,SOLICITUD,FECHA_INICIO,MULTAS_SIMIT,EDAD,CUOTA_INICIAL,GENERO,ESTADO_CIVIL,NIVEL_ESTUDIO,TIPO_VIVIENDA,PERSONAS_CARGO,...,TIPO_CLIENTE,HABITO_PAGO,ENDEUDAMIENTO,CALIFICACION,CREDITOS_VIGENTES,Puntaje Riesgo,TIPO_CONTRATO,ANTIGUEDAD,Marca producto,Estado
0,1,2017-01-02,0.0,29,0,F,Divorciado,Tecnólogico,Arrendada,2,...,Independiente sin Establecimiento,B,85860.0,A,2,2190,Indefinido,10,Marca A,Aprobada
1,2,2017-01-02,0.0,60,1035000,F,Soltero,Tecnólogico,Arrendada,1,...,Independiente sin Establecimiento,E,1270940.0,C,9,2142,Por Obra o Labor,131,Marca A,Desistida
2,3,2017-01-02,0.0,61,472500,M,Divorciado,Tecnólogico,Arrendada,1,...,Independiente sin Establecimiento,E,664620.0,C,7,2073,Indefinido,63,Marca B,Anulada
3,4,2017-01-02,0.0,25,0,M,Casado,Tecnólogico,Arrendada,2,...,Independiente sin Establecimiento,B,51940.0,A,2,1929,Otra,12,Marca A,Negada
4,5,2017-01-02,0.0,30,3672000,F,Divorciado,Tecnólogico,Arrendada,3,...,Independiente sin Establecimiento,B,0.0,A,2,3,INDEPENDIENTE SIN ESTABLECIMIENTO,131,Marca A,Anulada


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33668 entries, 0 to 33667
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   SOLICITUD          33668 non-null  int64         
 1   FECHA_INICIO       33668 non-null  datetime64[ns]
 2   MULTAS_SIMIT       33668 non-null  float64       
 3   EDAD               33668 non-null  int64         
 4   CUOTA_INICIAL      33668 non-null  int64         
 5   GENERO             33668 non-null  object        
 6   ESTADO_CIVIL       33668 non-null  object        
 7   NIVEL_ESTUDIO      33668 non-null  object        
 8   TIPO_VIVIENDA      33668 non-null  object        
 9   PERSONAS_CARGO     33668 non-null  int64         
 10  ZONA               33668 non-null  object        
 11  PLAZO              33668 non-null  int64         
 12  CODEUDOR           33668 non-null  object        
 13  VALOR_SOLICITADO   33668 non-null  float64       
 14  INGRES

In [14]:
df.isna().mean().sort_values(ascending=False).head(15)

TIPO_CONTRATO       0.011198
FECHA_INICIO        0.000000
SOLICITUD           0.000000
EDAD                0.000000
CUOTA_INICIAL       0.000000
GENERO              0.000000
ESTADO_CIVIL        0.000000
NIVEL_ESTUDIO       0.000000
TIPO_VIVIENDA       0.000000
PERSONAS_CARGO      0.000000
MULTAS_SIMIT        0.000000
ZONA                0.000000
PLAZO               0.000000
VALOR_SOLICITADO    0.000000
CODEUDOR            0.000000
dtype: float64

In [15]:
#Le ponemos un estándar al nombre de las columnas
df.columns= (
    df.columns.str.lower()
            .str.strip()
            .str.replace(" ","_")
)
# Vamos a normalizar columnas
# Fechas
df['fecha_inicio'] = pd.to_datetime(df['fecha_inicio'], errors= 'coerce')

# Columnas númericas
num_cols = [
    "ingresos", "egresos", "cuota_inicial", "personas_cargo",
    "valor_solicitado", "plazo", "edad",
    "multas_simit", "endeudamiento",
    "creditos_vigentes","antiguedad","puntaje_riesgo"
]

for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

cat_cols = [
    "genero", "estado_civil", "nivel_estudio","codeudor", "subtipo_cliente", "tipo_cliente",
    "tipo_vivienda", "estado", "habito_pago", "zona", "tipo_contrato","calificacion","marca_producto"
]

# las columnas que se nos muestran como object, las reemplezamos con string, quitamos espacios y las hacemos en mayúsculas.
for c in cat_cols:
    df[c] = (
        df[c].astype('string')
            .str.strip()
            .str.upper()
    )

# Ahora vamos a hacer manejo de nulos, imputarlos con la moda
df['tipo_contrato'] = df['tipo_contrato'].fillna(df['tipo_contrato'].mode()[0])

#Enriquecemos el df, a partir de lo que ya tenemos.
df['anio'] = df['fecha_inicio'].dt.year
df['mes'] = df['fecha_inicio'].dt.month
df['trimestre'] = df['fecha_inicio'].dt.quarter

#Validamos los cambios
print("Tipos\n-----")
print(df.dtypes.head(15))
print("\nNulos (%) después limpieza\n--------------------------")
print(df.isna().mean().sort_values(ascending=False).head(10))

Tipos
-----
solicitud                    int64
fecha_inicio        datetime64[ns]
multas_simit               float64
edad                         int64
cuota_inicial                int64
genero              string[python]
estado_civil        string[python]
nivel_estudio       string[python]
tipo_vivienda       string[python]
personas_cargo               int64
zona                string[python]
plazo                        int64
codeudor            string[python]
valor_solicitado           float64
ingresos                   float64
dtype: object

Nulos (%) después limpieza
--------------------------
solicitud         0.0
fecha_inicio      0.0
multas_simit      0.0
edad              0.0
cuota_inicial     0.0
genero            0.0
estado_civil      0.0
nivel_estudio     0.0
tipo_vivienda     0.0
personas_cargo    0.0
dtype: float64


In [20]:
from pathlib import Path

# crear la carpeta si no existe
Path("../data/processed").mkdir(parents=True, exist_ok=True)

# guardar parquet
df.to_parquet("../data/processed/creditos.parquet", index=False)
